In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [2]:
url = "http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario?codEstDE=&codEstATE=&tpEst=&nmEst=&nmRio=&codSubBacia=&codBacia=&nmMunicipio=&nmEstado=&sgResp=&sgOper=&telemetrica="
response = requests.get(url=url)

In [3]:
stations = pd.read_xml(response.content, xpath=".//Table")

In [5]:
stations_geo = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.Longitude, stations.Latitude))

In [6]:
# filter outliers
stations_geo = stations_geo[stations_geo.Longitude > -100]

In [7]:
# get Brazilian boundaries
brazil = gpd.read_file("/pfs/work7/workspace/scratch/tu_zxobe27-master_thesis/data/misc/gadm_410-BRA.geojson", engine="pyogrio")

In [14]:
stations_geo["in_bounds"] = stations_geo.intersects(brazil.iloc[0].geometry)

In [26]:
stations_geo.to_feather("/pfs/work7/workspace/scratch/tu_zxobe27-master_thesis/data/water_quality/stations.feather")

---

In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [2]:
stations_geo = gpd.read_feather("/pfs/work7/workspace/scratch/tu_zxobe27-master_thesis/data/water_quality/stations.feather")

In [3]:
stations_geo.loc[stations_geo["TipoEstacaoQualAgua"] == 1].dropna(subset = ["PeriodoTelemetricaInicio", "PeriodoTelemetricaFim"]).loc[:,"Codigo"]

21718    10300000
21761    12150000
21763    12180000
21772    12360000
21778    12400000
           ...   
35446    66520000
35467    66750000
36306    75450000
36309    75500000
36964    83301000
Name: Codigo, Length: 267, dtype: int64

In [9]:
url = f"http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica?codEstacao={10300000}&dataInicio=&dataFim=&tipoDados=2&nivelConsistencia="	
response = requests.get(url=url)

In [10]:
df = pd.read_xml(response.content, xpath=".//SerieHistorica")

In [14]:
df.columns

Index(['id', 'rowOrder', 'EstacaoCodigo', 'NivelConsistencia', 'DataHora',
       'MediaDiaria', 'TipoMedicaoCotas', 'Maxima', 'Minima', 'Media',
       'DiaMaxima', 'DiaMinima', 'MaximaStatus', 'MinimaStatus', 'MediaStatus',
       'MediaAnualStatus', 'Cota01', 'Cota02', 'Cota03', 'Cota04', 'Cota05',
       'Cota06', 'Cota07', 'Cota08', 'Cota09', 'Cota10', 'Cota11', 'Cota12',
       'Cota13', 'Cota14', 'Cota15', 'Cota16', 'Cota17', 'Cota18', 'Cota19',
       'Cota20', 'Cota21', 'Cota22', 'Cota23', 'Cota24', 'Cota25', 'Cota26',
       'Cota27', 'Cota28', 'Cota29', 'Cota30', 'Cota31', 'DataIns',
       'Cota01Status', 'Cota02Status', 'Cota03Status', 'Cota04Status',
       'Cota05Status', 'Cota06Status', 'Cota07Status', 'Cota08Status',
       'Cota09Status', 'Cota10Status', 'Cota11Status', 'Cota12Status',
       'Cota13Status', 'Cota14Status', 'Cota15Status', 'Cota16Status',
       'Cota17Status', 'Cota18Status', 'Cota19Status', 'Cota20Status',
       'Cota21Status', 'Cota22Status', 'Cot

In [13]:
df

,id,rowOrder,EstacaoCodigo,NivelConsistencia,DataHora,MediaDiaria,TipoMedicaoCotas,Maxima,Minima,Media,...,Cota23Status,Cota24Status,Cota25Status,Cota26Status,Cota27Status,Cota28Status,Cota29Status,Cota30Status,Cota31Status,MediaAnual
0,SerieHistorica1,0,10300000,1,2020-01-01 00:00:00,1,1,1472.0,1402.0,1459.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SerieHistorica2,1,10300000,2,2020-01-01 00:00:00,1,1,1472.0,1404.0,1458.83875,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,SerieHistorica3,2,10300000,1,2020-01-01 07:00:00,0,1,1472.0,1402.0,1459.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SerieHistorica4,3,10300000,1,2020-01-01 17:00:00,0,1,1472.0,1402.0,1459.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SerieHistorica5,4,10300000,1,2019-12-01 00:00:00,1,1,1479.0,1354.0,1455.00000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,SerieHistorica1248,1247,10300000,1,1980-11-01 00:00:00,1,1,1014.0,432.0,752.06670,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN
1248,SerieHistorica1249,1248,10300000,1,1980-10-01 00:00:00,1,1,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1249,SerieHistorica1250,1249,10300000,2,1980-10-01 00:00:00,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1250,SerieHistorica1251,1250,10300000,2,1980-09-01 00:00:00,1,1,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN


In [12]:
df.columns

Index(['id', 'rowOrder', 'EstacaoCodigo', 'NivelConsistencia', 'DataHora',
       'MediaDiaria', 'TipoMedicaoCotas', 'Maxima', 'Minima', 'Media',
       'DiaMaxima', 'DiaMinima', 'MaximaStatus', 'MinimaStatus', 'MediaStatus',
       'MediaAnualStatus', 'Cota01', 'Cota02', 'Cota03', 'Cota04', 'Cota05',
       'Cota06', 'Cota07', 'Cota08', 'Cota09', 'Cota10', 'Cota11', 'Cota12',
       'Cota13', 'Cota14', 'Cota15', 'Cota16', 'Cota17', 'Cota18', 'Cota19',
       'Cota20', 'Cota21', 'Cota22', 'Cota23', 'Cota24', 'Cota25', 'Cota26',
       'Cota27', 'Cota28', 'Cota29', 'Cota30', 'Cota31', 'DataIns',
       'Cota01Status', 'Cota02Status', 'Cota03Status', 'Cota04Status',
       'Cota05Status', 'Cota06Status', 'Cota07Status', 'Cota08Status',
       'Cota09Status', 'Cota10Status', 'Cota11Status', 'Cota12Status',
       'Cota13Status', 'Cota14Status', 'Cota15Status', 'Cota16Status',
       'Cota17Status', 'Cota18Status', 'Cota19Status', 'Cota20Status',
       'Cota21Status', 'Cota22Status', 'Cot

In [18]:
stations_geo[stations_geo.in_bounds]

,id,rowOrder,BaciaCodigo,SubBaciaCodigo,RioCodigo,RioNome,EstadoCodigo,nmEstado,MunicipioCodigo,nmMunicipio,...,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando,Descricao,NumImagens,DataIns,DataAlt,geometry,in_bounds
0,Table1,0,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,...,0.0,0.0,2024-02-21 00:00:00,0,None,0.0,None,2024-02-21 00:00:00,POINT (-47.55000 -0.65000),True
1,Table2,1,3,32,NaN,None,5,PARÁ,5043000,MARACANÃ,...,0.0,0.0,2005-03-29 00:00:00,1,None,0.0,None,None,POINT (-47.50000 -0.83333),True
2,Table3,2,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,...,0.0,0.0,2024-01-15 00:00:00,1,AUTOMÁTICA - IRIS-1,1.0,None,2024-01-15 00:00:00,POINT (-47.35360 -0.62310),True
3,Table4,3,3,32,NaN,None,5,PARÁ,5029000,CURUÇA,...,0.0,0.0,2018-06-18 00:00:00,1,AUTOMÁTICA - AGSOLV,2.0,None,2018-06-18 00:00:00,POINT (-47.85360 -0.73750),True
4,Table5,4,3,32,NaN,None,5,PARÁ,5061000,PRIMAVERA,...,0.0,0.0,2023-11-07 00:00:00,1,None,1.0,None,2023-11-07 00:00:00,POINT (-47.09940 -0.92940),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38055,Table38056,38055,8,88,88100000.0,CANAL DE SÃO GONÇALO,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,0.0,1.0,2018-02-02 00:00:00,0,None,0.0,None,2018-02-02 00:00:00,POINT (-52.33330 -31.78220),True
38056,Table38057,38056,8,88,88100000.0,CANAL DE SÃO GONÇALO,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,0.0,0.0,2017-03-28 00:00:00,0,None,0.0,None,2017-03-28 00:00:00,POINT (-52.22810 -31.78640),True
38057,Table38058,38057,8,88,88105000.0,ARROIO PELOTAS,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,3.0,0.0,2018-02-01 00:00:00,1,None,0.0,2015-04-27 00:00:00,2018-02-01 00:00:00,POINT (-52.28310 -31.75750),True
38058,Table38059,38058,8,88,88105000.0,ARROIO PELOTAS,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,3.0,1.0,2018-12-06 00:00:00,1,None,0.0,None,2018-12-06 00:00:00,POINT (-52.46250 -31.57360),True


In [33]:
stations_geo.within(brazil.unary_union)

In [30]:
stations_geo[stations_geo.intersects(brazil.geometry.iloc[0])].plot()

KeyboardInterrupt: 